In [ ]:
import MLJ
import DataFrames as DF
import CSV
using DataFramesMeta
using CairoMakie
using LaTeXStrings

In [ ]:
include("PreprocessAerosolData.jl")
include("ActivationEmulatorModels.jl")

In [ ]:
function plot_single_var_profile(gp, dataset_filename::String, x_axis_var::Symbol, mach_filenames, mach_labels; kwargs...)
    X_test, Y_test = read_aerosol_dataset(dataset_filename)
    ARG_S_max = exp.(preprocess_aerosol_data(X_test).log_ARG_S_max)
    PySDM_S_max = exp.(Y_test)
    x_range = X_test[!, x_axis_var]
    ax = Axis(gp; yscale=log10, xlabel=String(x_axis_var), ylabel="Predicted / actual S_max", kwargs...)
    lines!(ax, x_range, fill(1, size(x_range)), color=:black)
    # lines!(ax, x_range, PySDM_S_max, label="PySDM")
    lines!(ax, x_range, ARG_S_max ./ PySDM_S_max, label="ARG")
    for (mach_filename, mach_label) in zip(mach_filenames, mach_labels)
        mach = MLJ.machine(mach_filename)
        pred_S_max = exp.(MLJ.predict(mach, X_test))
        lines!(ax, x_range, pred_S_max ./ PySDM_S_max, label=mach_label)
    end
    return ax
end

In [ ]:
mach_filenames = ["emulators/1modal_evotree_machine1.jls", "emulators/1modal_nn_machine1.jls", "emulators/1modal_gp_machine1.jls"]
mach_labels = ["EvoTree model", "NN model", "GP model"]
fig = Figure()
ax = plot_single_var_profile(fig[1,1], "datasets/single_var_profiles/Silva_Ma_Fig7_mode_N.csv", :mode_1_N, mach_filenames, mach_labels, xscale=log10)
plot_single_var_profile(fig[1,2], "datasets/single_var_profiles/Silva_Ma_Fig7_mode_mean.csv", :mode_1_mean, mach_filenames, mach_labels, xscale=log10)
plot_single_var_profile(fig[2,1], "datasets/single_var_profiles/Silva_Ma_Fig7_velocity.csv", :velocity, mach_filenames, mach_labels, xscale=log10)
plot_single_var_profile(fig[2,2], "datasets/single_var_profiles/Silva_Ma_Fig7_kappa.csv", :mode_1_kappa, mach_filenames, mach_labels)
Legend(fig[1,3], ax)
fig

In [ ]:
mach_filenames = ["emulators/2modal_evotree_machine1.jls", "emulators/2modal_nn_machine1.jls", "emulators/2modal_gp_machine1.jls"]
mach_labels = ["EvoTree model", "NN model", "GP model"]
fig = Figure()
ax = plot_single_var_profile(fig[1,1], "datasets/single_var_profiles/ARG2000_Fig1_mode_2_N.csv", :mode_2_N, mach_filenames, mach_labels, xscale=log10)
plot_single_var_profile(fig[1,2], "datasets/single_var_profiles/ARG2000_Fig4_mode_2_mean.csv", :mode_2_mean, mach_filenames, mach_labels, xscale=log10)
plot_single_var_profile(fig[2,1], "datasets/single_var_profiles/ARG2000_Fig7_velocity.csv", :velocity, mach_filenames, mach_labels, xscale=log10)
Legend(fig[1,3], ax)
fig